**🎯 Objetivo del Script**

Recopilar información de videojuegos lanzados entre 1995 y 2024 desde la API de RAWG, incluyendo solo aquellos con puntuación válida y géneros, y guardar los resultados en un archivo .csv llamado videojuegos_1995_2024_completo.csv.

**1. Importación de librerías**




In [1]:
import requests
import time
import csv

"Estas son las librerías básicas utilizadas:"


**requests:** *para enviar peticiones HTTP a la API de RAWG.*

**time:** *permite pausar la ejecución entre peticiones para evitar ser bloqueado por exceso de solicitudes.*

**csv:** *para guardar los datos recolectados en un archivo CSV.*



**2. Definición de constantes**

In [ ]:
API_KEY = '9e9ab06256174d82a01dc73f02418ca2'
BASE_URL = 'https://api.rawg.io/api/games'

**API_KEY:** tu clave personal de la API RAWG (funciona correctamente si es válida).

**BASE_URL:** la dirección base para acceder al listado de videojuegos en la API.

3. **Función principal:** *obtener_juegos_desde_1995()*


In [ ]:
def obtener_juegos_desde_1995(paginas_por_año=3):

Esta función es responsable de:

Recorrer cada año entre 1995 y 2024.

Descargar hasta paginas_por_año (por defecto 3) páginas por año.

Almacenar solo juegos con puntuación válida y géneros.

Asegurar que todos los años estén representados, aunque no tengan juegos válidos.

**4. Inicializar la lista de resultados**

In [ ]:
juegos_por_año = []

Esta lista contendrá todos los datos recolectados, año a año.



**5. Recorrido año por año**

In [ ]:
for año in range(1995, 2025):

Recorre todos los años desde 1995 hasta 2024.

*range(1995, 2025)* incluye 2024 porque el límite superior no se incluye.

**6. Recorrido de páginas por cada año**



In [ ]:
for pagina in range(1, paginas_por_año + 1):

RAWG devuelve máximo 40 juegos por página.

Este bucle recorre varias páginas por año (por defecto 3 = hasta 120 juegos/año).

**7. Parámetros de consulta para la API**

In [ ]:
params = {
    'key': API_KEY,
    'dates': f'{año}-01-01,{año}-12-31',
    'page': pagina,
    'page_size': 40,
    'ordering': '-rating'
}

Estos parámetros:

Filtran los resultados por fecha de lanzamiento.

Limitan a 40 juegos por página.

Ordenan los juegos de mayor a menor puntuación (-rating).

**8. Petición a la API**


In [ ]:
response = requests.get(BASE_URL, params=params)

Envía la solicitud a la API para obtener los juegos de esa página del año correspondiente.

**9. Manejo de errores**

In [ ]:
if response.status_code != 200:
    print(f"Error {response.status_code} en el año {año}, página {pagina}")
    break

Si hay un error (como una clave inválida o límite excedido), lo reporta y detiene el bucle de esa página

**10. Procesamiento de la respuesta JSON**


In [ ]:
data = response.json()

Convierte la respuesta HTTP en un diccionario de Python (dict) para trabajar con los datos.

**11. Filtrar juegos válidos**

In [ ]:
for juego in data.get('results', []):
    rating = juego.get('rating')
    if rating and rating > 0 and juego.get('genres'):

Itera por cada juego de la página.

Se asegura de que:

El juego tiene puntuación (rating > 0).

El juego tiene al menos un género.

**12. Guardar los juegos válidos**


In [ ]:
juegos_este_año.append({
    'nombre': juego.get('name'),
    'fecha_lanzamiento': juego.get('released'),
    'año': año,
    'puntuacion_media': rating,
    'generos': ', '.join([g['slug'] for g in juego['genres']])

Extrae la información útil del juego:

Nombre, fecha, año, puntuación, géneros.

Convierte los géneros (lista de diccionarios) en una cadena separada por comas.

**13. Comprobar si hay más páginas**


In [ ]:
if not data.get('next'):
    break

La API incluye un campo next que indica si hay otra página disponible.

Si no lo hay, se sale del bucle de páginas.

**14. Pausa entre peticiones**



In [ ]:
time.sleep(1)

Espera 1 segundo antes de la siguiente petición para evitar sobrecargar la API (buena práctica contra “rate limit”).

**15. Registrar año sin resultados (si aplica)**



In [ ]:
if not juegos_este_año:
    juegos_por_año.append({
        'nombre': 'Sin datos',
        'fecha_lanzamiento': None,
        'año': año,
        'puntuacion_media': None,
        'generos': 'sin_datos'
    })

Si no se encontraron juegos válidos, se añade una fila “vacía” indicando que ese año no tuvo datos.

**16. Agregar los juegos válidos al conjunto final**


In [ ]:
else:
    juegos_por_año.extend(juegos_este_año)

Si hubo juegos válidos, se añaden a la lista global juegos_por_año.



**17. Retornar la lista final**


In [ ]:
return juegos_por_año

Devuelve todos los juegos recopilados (válidos o con “sin datos”).



**18. Función para guardar en CSV**


In [ ]:
def guardar_csv(datos, archivo='videojuegos_1995_2024_completo.csv'):

Abre un archivo CSV y guarda los datos en columnas bien definidas.

In [ ]:
campos = ['nombre', 'fecha_lanzamiento', 'año', 'puntuacion_media', 'generos']
writer = csv.DictWriter(f, fieldnames=campos)
writer.writeheader()
writer.writerows(datos)

Escribe la cabecera y luego todas las filas del listado de videojuegos.



**19. Bloque de ejecución principal**



In [ ]:
if __name__ == '__main__':
    juegos = obtener_juegos_desde_1995(paginas_por_año=3)
    guardar_csv(juegos)

Ejecuta la descarga de los juegos.

Guarda el resultado en un archivo CSV llamado: *videojuegos_1995_2024_completo.csv*